This notebooks takes the pickle file of filtered VCF files (SNPS) and reduces the feature space with a Random Forest algorithm.

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# from keras.models import Sequential
# from keras.layers import Dense,Dropout, Flatten,BatchNormalization, GaussianNoise
import matplotlib.pyplot as plt
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
# from tensorflow.keras import initializers
# from tensorflow.keras import regularizers
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import os

In [2]:
os.chdir("/users/anair27/data/anair27")

Import the outcome variable from the clinical spreadsheet.

In [3]:
clinical = pd.read_csv("./singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_clinical/processed_clinical.csv")

In [4]:
diagnosis = clinical[["vital_status_Dead", "case_id"]]
diagnosis.head(10)

,vital_status_Dead,case_id
0,0,TCGA-78-8640
1,0,TCGA-05-5425
2,0,TCGA-55-7815
3,1,TCGA-44-7661
4,0,TCGA-97-7554
5,1,TCGA-J2-A4AD
6,1,TCGA-55-6981
7,0,TCGA-86-8281
8,1,TCGA-44-A4SU
9,1,TCGA-93-A4JO


In [5]:
def read_gene_expression(filepath, case_id):
    arr = []
    with open(filepath) as f:
        lines = f.readlines()
        for l in lines:
            arr.append(l.upper().split())
    matrix = pd.DataFrame(arr)[1:]
    matrix.columns = matrix.iloc[0]
    matrix = matrix[matrix["GENE_TYPE"] == "PROTEIN_CODING"]
    matrix[['GENE_ID', 'FPKM_UNSTRANDED']]
    matrix = matrix[matrix["GENE_TYPE"] == "PROTEIN_CODING"]
    matrix = matrix[['GENE_ID', 'FPKM_UNSTRANDED']].set_index('GENE_ID').transpose()
    return matrix.rename(columns={'GENE_ID': 'CASE_ID'},index={'FPKM_UNSTRANDED': case_id}).reset_index().rename(columns={1:'CASE_ID'})
    

In [6]:
cases = os.listdir("data_by_cases")
cases[0:10]

['TCGA-35-4122',
 'TCGA-75-6203',
 'TCGA-75-5146',
 'TCGA-78-8648',
 'TCGA-55-A4DG',
 'TCGA-MP-A4SY',
 'TCGA-67-3771',
 'TCGA-44-A479',
 'TCGA-78-7156',
 'TCGA-55-7724']

In [7]:
gene_exp_data = []
for case in cases:
    contents_gene_exp = os.listdir(os.path.join("data_by_cases", case, "gene_expression"))
    if len(contents_gene_exp) == 0:
        print(f"{case} has no gene expression data")
    else:
        filename = contents_gene_exp[0]
        path = os.path.join("data_by_cases", case, "gene_expression", filename)
        
        gene_exp_data.append(read_gene_expression(path, case))
# pd.concat(gene_exp_data, axis = 0)        

TCGA-05-4245 has no gene expression data
TCGA-44-2664 has no gene expression data
TCGA-67-3776 has no gene expression data
TCGA-44-A47F has no gene expression data
TCGA-MP-A4T2 has no gene expression data
TCGA-55-8615 has no gene expression data


In [8]:
len(gene_exp_data)

515

In [9]:
all_gene_exp = pd.concat(gene_exp_data, axis = 0)
all_gene_exp

GENE_ID,CASE_ID,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,...,ENSG00000288649.1,ENSG00000288654.1,ENSG00000288656.1,ENSG00000288658.1,ENSG00000288660.1,ENSG00000288661.1,ENSG00000288669.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
0,TCGA-35-4122,46.9281,0.4808,57.9971,1.5523,3.6818,9.8975,6.2302,25.2810,7.0157,...,0.0000,0.0000,0.0000,0.7707,0.0000,0.0000,0.0000,0.0000,0.0061,0.1877
0,TCGA-75-6203,11.7451,0.0000,18.2141,1.8852,0.5896,25.3843,9.8348,17.5786,1.7791,...,0.0000,0.0000,0.0077,0.0302,0.0000,0.0000,0.0000,0.0000,0.0279,0.2131
0,TCGA-75-5146,20.3682,0.0225,31.1076,3.2412,0.9620,3.9068,11.0951,22.6927,4.2451,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0104,0.0988
0,TCGA-78-8648,5.6365,0.0000,19.6206,1.4938,0.8152,26.8137,18.2827,20.7673,3.5553,...,0.0000,0.0000,0.0000,0.2541,0.0000,0.0000,0.0000,0.0000,0.0107,0.2448
0,TCGA-55-A4DG,4.6527,0.0000,30.7542,5.0779,1.0228,3.8962,6.0109,12.6130,11.5452,...,0.0000,0.0000,0.0000,0.2708,0.1105,0.0000,0.0000,0.0000,0.0362,0.2391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,TCGA-64-5775,17.6945,0.1976,22.4199,0.8574,0.6465,5.0291,3.9113,12.8731,0.6119,...,0.0000,0.0000,0.0000,0.0227,0.0000,0.0000,0.0000,0.0000,0.0093,0.0801
0,TCGA-05-4418,24.3521,0.0098,18.7905,1.9846,1.0234,5.1553,7.5910,30.2022,48.3523,...,0.0000,0.0000,0.0000,0.2720,0.0000,0.0000,0.0000,0.0000,0.0135,0.1458
0,TCGA-62-8398,17.9135,0.0000,24.8826,2.5163,2.3549,4.9414,18.4484,21.9511,10.1748,...,0.0000,0.0000,0.0047,0.0435,0.0000,0.0000,0.0046,0.0000,0.0172,0.1206
0,TCGA-55-8097,9.1147,0.0099,16.1424,2.6900,0.6635,5.2058,8.3084,6.7483,1.2258,...,0.0000,0.0000,0.0000,0.0692,0.0404,0.0000,0.0000,0.0000,0.0244,0.1659


In [16]:
all_gene_exp = all_gene_exp.rename(columns={"CASE_ID":"case_id"})

In [17]:
all_gene_exp.to_csv("singh-lab-TCGA-project/MADDi_model/data_processed/transcriptomic_data.csv")

In [11]:
all_gene_exp.to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/all_gene_exp_FPKM.pkl")

---


In [39]:
clinical = pd.read_csv("./singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_clinical/processed_clinical.csv")

In [40]:
diagnosis = clinical[["vital_status_Dead", "case_id"]]
diagnosis.head(10)

,vital_status_Dead,case_id
0,0,TCGA-78-8640
1,0,TCGA-05-5425
2,0,TCGA-55-7815
3,1,TCGA-44-7661
4,0,TCGA-97-7554
5,1,TCGA-J2-A4AD
6,1,TCGA-55-6981
7,0,TCGA-86-8281
8,1,TCGA-44-A4SU
9,1,TCGA-93-A4JO


In [41]:
gene_exp = pd.read_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/all_gene_exp_FPKM.pkl")
gene_exp = gene_exp.rename(columns = {"CASE_ID":"case_id"})


In [42]:
gene_exp_merge = gene_exp.merge(diagnosis, on = "case_id").drop_duplicates()

In [27]:
# gene_exp_merge.to_csv("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/all_gene_exp_FPKM.csv")

In [43]:
gene_exp_merge.head(10)

,case_id,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,...,ENSG00000288654.1,ENSG00000288656.1,ENSG00000288658.1,ENSG00000288660.1,ENSG00000288661.1,ENSG00000288669.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1,vital_status_Dead
0,TCGA-35-4122,46.9281,0.4808,57.9971,1.5523,3.6818,9.8975,6.2302,25.2810,7.0157,...,0.0000,0.0000,0.7707,0.0000,0.0000,0.0000,0.0000,0.0061,0.1877,0
1,TCGA-75-6203,11.7451,0.0000,18.2141,1.8852,0.5896,25.3843,9.8348,17.5786,1.7791,...,0.0000,0.0077,0.0302,0.0000,0.0000,0.0000,0.0000,0.0279,0.2131,0
2,TCGA-75-5146,20.3682,0.0225,31.1076,3.2412,0.9620,3.9068,11.0951,22.6927,4.2451,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0104,0.0988,0
3,TCGA-78-8648,5.6365,0.0000,19.6206,1.4938,0.8152,26.8137,18.2827,20.7673,3.5553,...,0.0000,0.0000,0.2541,0.0000,0.0000,0.0000,0.0000,0.0107,0.2448,1
4,TCGA-55-A4DG,4.6527,0.0000,30.7542,5.0779,1.0228,3.8962,6.0109,12.6130,11.5452,...,0.0000,0.0000,0.2708,0.1105,0.0000,0.0000,0.0000,0.0362,0.2391,0
5,TCGA-MP-A4SY,18.3232,0.0282,37.3027,2.1270,1.7163,4.6491,13.0511,11.7705,4.3714,...,0.0000,0.0000,0.1685,0.0000,0.0000,0.0000,0.0000,0.0216,0.1240,1
6,TCGA-67-3771,7.8585,0.0229,50.9605,2.9411,1.9223,9.0286,4.5518,19.5891,51.1632,...,0.0000,0.0000,0.8899,0.0000,0.0000,0.0000,0.0000,0.0106,0.1814,0
7,TCGA-44-A479,12.2301,0.0000,47.2267,2.3381,1.4807,7.6328,22.0589,24.0394,8.6965,...,0.0000,0.0075,0.1683,0.0000,0.0000,0.0000,0.0000,0.0092,0.3846,0
8,TCGA-78-7156,14.0995,0.0377,21.9471,4.8401,0.6579,1.8943,36.7351,10.3247,8.8594,...,0.0000,0.0000,0.0282,0.0000,0.0000,0.0000,0.0000,0.0434,0.0829,1
9,TCGA-55-7724,17.1607,0.0315,32.9382,2.4359,1.0502,8.7615,18.0248,19.2064,10.1964,...,0.0000,0.0000,0.6575,0.0000,0.0000,0.0000,0.0000,0.0097,0.0553,0


In [44]:
cols = list(set(gene_exp_merge.columns) - set(["vital_status_Dead", "case_id"]))
# X = pd.DataFrame(gene_exp_merge[cols].values.astype(float))
# y = gene_exp_merge["vital_status_Dead"].astype(int).values
X = gene_exp_merge[cols].astype(float)
y = gene_exp_merge["vital_status_Dead"].astype(int)

In [45]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(X, y)
print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

Original dataset shape Counter({0: 328, 1: 187})
Resample dataset shape Counter({0: 328, 1: 328})


In [46]:
X_train, X_test, y_train, y_test = train_test_split(x_ros, y_ros, test_size=0.1)

In [48]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [49]:
(sel.get_support())

array([False, False, False, ..., False, False, False])

In [50]:
X_train

,ENSG00000237763.9,ENSG00000186862.20,ENSG00000184954.4,ENSG00000118181.11,ENSG00000213903.9,ENSG00000158560.14,ENSG00000121579.13,ENSG00000129484.14,ENSG00000165821.12,ENSG00000139977.14,...,ENSG00000144445.17,ENSG00000116191.17,ENSG00000150051.14,ENSG00000256980.5,ENSG00000186980.7,ENSG00000153292.16,ENSG00000167552.14,ENSG00000101294.18,ENSG00000101443.18,ENSG00000185507.21
352,0.0168,0.5214,0.0000,292.4626,2.0165,0.1332,29.5972,7.3214,1.7818,6.2672,...,1.4264,4.1619,0.1891,0.0000,0.0,0.1102,51.4454,31.9675,349.6679,15.0015
39,0.0262,0.1609,0.0000,464.0114,1.3420,0.1177,26.0807,4.5190,3.6073,2.7417,...,1.3760,5.8201,0.9131,0.0000,0.0,44.3231,27.1144,19.4730,136.9895,14.5071
645,0.0087,0.0560,0.0000,263.7711,1.5632,0.7045,43.0391,6.0115,1.3976,3.3205,...,1.4115,4.6554,0.7995,0.0913,0.0,0.8077,40.1562,21.7392,77.3017,22.0105
163,0.0000,0.2206,0.0287,226.1991,0.8700,0.8031,23.4604,2.5701,3.4378,5.1961,...,2.9006,3.4242,0.2430,0.0000,0.0,1.0086,88.3379,26.3481,125.0993,13.8907
137,0.0108,0.4589,0.0000,362.8637,2.9204,0.0829,38.5081,5.4213,3.0154,3.1365,...,1.8101,4.9667,0.4413,0.0284,0.0,24.1562,46.7088,19.9025,160.1505,17.8766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,0.0000,0.0745,0.0000,499.4135,2.3764,0.4230,21.0737,4.0394,2.5259,7.3400,...,1.7198,3.4855,0.9282,0.1106,0.0,0.2837,45.3555,38.8721,402.7980,18.2466
290,0.0000,0.3617,0.0000,374.2604,2.9310,0.7077,25.5315,5.1753,1.8752,3.3417,...,2.9830,6.3227,0.1650,0.0109,0.0,7.0997,52.1877,24.8226,22.7613,42.1260
567,0.0101,0.4396,0.0000,502.9701,0.8930,0.3315,38.7746,4.5476,0.8485,1.0369,...,0.9716,4.3361,0.6267,0.0000,0.0,31.2027,14.9127,20.6641,69.2043,7.3137
7,0.0000,0.4092,0.0000,305.5867,3.0584,0.5593,28.1406,6.3840,3.1152,7.8626,...,3.4439,5.2605,0.1287,0.0000,0.0,0.1613,44.8924,36.0446,116.1288,25.8260


In [51]:
selected_feat= pd.DataFrame(X_train).columns[(sel.get_support())]
len(selected_feat)

3878

In [52]:
# feature reduction by RF classifier
selected_feat


Index(['ENSG00000121579.13', 'ENSG00000187912.12', 'ENSG00000198355.5',
       'ENSG00000077713.19', 'ENSG00000162755.14', 'ENSG00000179152.20',
       'ENSG00000277639.2', 'ENSG00000266173.7', 'ENSG00000165182.12',
       'ENSG00000242616.4',
       ...
       'ENSG00000107651.13', 'ENSG00000198807.13', 'ENSG00000137821.12',
       'ENSG00000138867.17', 'ENSG00000215113.6', 'ENSG00000166068.13',
       'ENSG00000007129.18', 'ENSG00000111348.9', 'ENSG00000106992.19',
       'ENSG00000150051.14'],
      dtype='object', length=3878)

In [29]:
l = ["case_id", "vital_status_Dead"]
l.extend(selected_feat)

In [30]:
#saving the features with the old dataframe so that the overlap test set can still be used when combining all data
gene_exp_merge[l].to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/all_gene_exp_select.pkl")

In [53]:
X_train[selected_feat].to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/gene_exp_X_train.pkl")
y_train.to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/gene_exp_y_train.pkl")

X_test[selected_feat].to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/gene_exp_X_test.pkl")
y_test.to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/gene_exp_y_test.pkl")

In [ ]:
#reading all the SNP files
vcf = pd.read_pickle("all_vcfs.pkl")

In [ ]:
#reading in the diagnosis data
m = pd.read_csv("diagnosis_full.csv").drop("Unnamed: 0", axis=1).rename(columns = {"Subject":"subject", "GROUP": "label"})

In [ ]:
#making sure all the diagnosis agree
m = m[m["label"] != -1]

In [ ]:
#merging SNPs with diagnosis
vcf = vcf.merge(m[["subject", "label"]], on = "subject")

In [ ]:
vcf = vcf.drop_duplicates()

In [ ]:
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv")

#removing ids from the overlap test set, saving it as a new variable
vcf1 = vcf[~vcf["subject"].isin(list(ts["subject"].values))]

In [ ]:
# Using Random Forest to reduce feature dimensions
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))

In [ ]:
cols = list(set(vcf1.columns) - set(["subject", "Group", "label"]))
X = vcf1[cols].values.astype(int)
y = vcf1["label"].astype(int).values

for i in range(len(y)):
    y[i] = y[i]-1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [ ]:
#fitting random forest only on the training data so that there is not influence on the testing performance
sel.fit(X_train, y_train)

In [ ]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

In [ ]:
print(selected_feat)

In [ ]:
l = ["label", "subject", "Group"]
l.extend(selected_feat)

In [ ]:
#saving the features with the old dataframe so that the overlap test set can still be used when combining all data
vcf[l].to_pickle("vcf_select.pkl")

In [ ]:
X_train[selected_feat].to_pickle("X_train_vcf.pkl")
y_train.to_pickle("y_train_vcf.pkl")

X_test[selected_feat].to_pickle("X_test_vcf.pkl")
y_test.to_pickle("y_test_vcf.pkl")